# Fetch Binance Data

In [7]:
import requests
from datetime import datetime, timedelta
import time

def get_binance_server_time():
    """
    Fetch the current server time from Binance.
    """
    url = 'https://fapi.binance.com/fapi/v1/time'
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch server time: {response.status_code} - {response.text}")
    return response.json()['serverTime']

def fetch_ohlc(symbol, interval, lookback_minutes):
    """
    Fetch OHLC data for the specified symbol and interval using the Binance API URL.
    
    :param symbol: Trading pair (e.g., 'BTC')
    :param interval: Kline interval (e.g., '1m' for 1 minute)
    :param lookback_minutes: Number of minutes to look back
    :return: List of OHLC data
    """
    # Get the current Binance server time
    server_time = get_binance_server_time()
    server_time_dt = datetime.fromtimestamp(server_time / 1000)
    
    # Calculate the start time based on the server time and the lookback period
    end_time = server_time_dt - timedelta(minutes=1)  # Use the previous minute as the end time
    start_time = end_time - timedelta(minutes=lookback_minutes)
    
    # Convert times to milliseconds
    start_time_ms = int(start_time.timestamp() * 1000)
    end_time_ms = int(end_time.timestamp() * 1000)
    
    # Binance API URL
    url = (
        f'https://fapi.binance.com/fapi/v1/klines'
        f'?symbol={symbol}USDT'
        f'&interval={interval}'
        f'&startTime={start_time_ms}'
        f'&endTime={end_time_ms}'
        f'&limit={lookback_minutes + 1}'  # +1 to include the current minute
    )
    
    # Fetch the data
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data: {response.status_code} - {response.text}")
    
    # Process the klines to extract OHLC data
    ohlc_data = []
    for kline in response.json():
        time_stamp = kline[0]
        open_price = float(kline[1])
        high_price = float(kline[2])
        low_price = float(kline[3])
        close_price = float(kline[4])
        ohlc_data.append({
            'time': datetime.fromtimestamp(time_stamp / 1000).strftime('%Y-%m-%d %H:%M:%S'),
            'open': open_price,
            'high': high_price,
            'low': low_price,
            'close': close_price
        })
    
    return ohlc_data

def get_minute_info(symbol, interval, lookback_minutes):
    """
    Fetch OHLC data every minute and calculate max close, min close, and newest close.
    """
    while True:
        # Wait until the 1st second of the next minute
        now = datetime.utcnow()
        time_to_wait = 60 - now.second + 1  # Wait until the 1st second of the next minute
        print(f"Waiting {time_to_wait} seconds until the 1st second of the next minute...")
        time.sleep(time_to_wait)
        
        # Add a 5-second delay to ensure the OHLC data is finalized
        print("Adding 5-second delay to ensure OHLC data is finalized...")
        time.sleep(5)
        
        # Fetch OHLC data
        ohlc_data = fetch_ohlc(symbol, interval, lookback_minutes)
        
        # Ensure we have exactly 101 candles (100 previous + 1 newest)
        if len(ohlc_data) > lookback_minutes + 1:
            ohlc_data = ohlc_data[:lookback_minutes + 1]  # Trim to 101 candles
        
        # Extract close prices
        close_prices = [candle['close'] for candle in ohlc_data]
        
        # Calculate max close, min close, and newest close
        max_close = max(close_prices[:-1])  # Max close of the previous 100 minutes
        min_close = min(close_prices[:-1])  # Min close of the previous 100 minutes
        newest_close = close_prices[-1]     # Close of the newest (101st) minute
        
        # Print the results
        print(f"Time: {ohlc_data[-1]['time']}")
        print(f"Max Close (Prev 100 Minutes): {max_close}")
        print(f"Min Close (Prev 100 Minutes): {min_close}")
        print(f"Newest Close: {newest_close}")
        print("-" * 40)

# Parameters
symbol = 'BTC'  # Symbol without 'USDT'
interval = '1m'  # 1-minute interval
lookback_minutes = 100  # Adjustable lookback period

# Start fetching data every minute
get_minute_info(symbol, interval, lookback_minutes)

/tmp/ipykernel_1871636/1020181667.py:75: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


Waiting 48 seconds until the 1st second of the next minute...
Adding 5-second delay to ensure OHLC data is finalized...
Time: 2025-02-28 05:04:00
Max Close (Prev 100 Minutes): 81134.4
Min Close (Prev 100 Minutes): 79598.5
Newest Close: 80333.7
----------------------------------------
Waiting 55 seconds until the 1st second of the next minute...
Adding 5-second delay to ensure OHLC data is finalized...
Time: 2025-02-28 05:05:00
Max Close (Prev 100 Minutes): 81134.4
Min Close (Prev 100 Minutes): 79598.5
Newest Close: 80043.7
----------------------------------------
Waiting 55 seconds until the 1st second of the next minute...


KeyboardInterrupt: 

In [16]:
import requests
from datetime import datetime, timedelta
import time

# Telegram Bot Configuration
TELEGRAM_BOT_TOKEN = "7238226122:AAFf2OAEarJUpM6Bmd20RojCgPG4TdNYrIA"  # Replace with your bot token
TELEGRAM_CHAT_ID = "-4790253567"      # Replace with your group chat ID

def send_telegram_message(message):
    """Send a message to a Telegram group using the bot."""
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    payload = {
        "chat_id": TELEGRAM_CHAT_ID,
        "text": message,
        "parse_mode": "Markdown"  # Use Markdown formatting
    }
    response = requests.post(url, json=payload)
    if response.status_code != 200:
        print(f"Failed to send Telegram message: {response.text}")
    else:
        print("Telegram message sent successfully!")

def get_binance_server_time():
    """Fetch the current server time from Binance."""
    url = 'https://fapi.binance.com/fapi/v1/time'
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch server time: {response.status_code} - {response.text}")
    return response.json()['serverTime']

def fetch_ohlc(symbol, interval, lookback_minutes):
    """Fetch OHLC data for the specified symbol and interval."""
    server_time = get_binance_server_time()
    server_time_dt = datetime.fromtimestamp(server_time / 1000)
    end_time = server_time_dt - timedelta(minutes=1)
    start_time = end_time - timedelta(minutes=lookback_minutes)
    
    url = (
        f'https://fapi.binance.com/fapi/v1/klines'
        f'?symbol={symbol}USDT&interval={interval}'
        f'&startTime={int(start_time.timestamp() * 1000)}'
        f'&endTime={int(end_time.timestamp() * 1000)}'
        f'&limit={lookback_minutes + 1}'
    )
    
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data: {response.status_code} - {response.text}")
    
    ohlc_data = []
    for kline in response.json():
        ohlc_data.append({
            'time': datetime.fromtimestamp(kline[0] / 1000).strftime('%Y-%m-%d %H:%M:%S'),
            'open': float(kline[1]),
            'high': float(kline[2]),
            'low': float(kline[3]),
            'close': float(kline[4]),
        })
    return ohlc_data

def get_minute_info(
    symbol, 
    interval, 
    lookback_minutes, 
    reversal_candles=10,
    tp_percent=3,         # Adjustable TP percentage (default: 3%)
    sl_percent=3,         # Adjustable SL percentage (default: 3%)
    time_limit_candles=1500  # Adjustable time limit (default: 1500 candles)
):
    """Fetch OHLC data and implement sequence, reversal, confirmation, TP/SL, and time limit logic."""
    sequence = None
    breakout_close = None
    confirmation_candles = []
    first_signal = None
    reversal_level = None
    reversal_candle_count = 0
    confirmed_signal = None
    long_zone = None
    short_zone = None
    entry_triggered = False
    tp_sl_levels = {}
    confirmed_time = None
    historical_events = {}
    candle_counter = 0

    while True:
        now = datetime.utcnow()
        time_to_wait = 60 - now.second + 1
        print(f"Waiting {time_to_wait} seconds until next minute...")
        time.sleep(time_to_wait)
        time.sleep(5)  # 5-second delay

        ohlc_data = fetch_ohlc(symbol, interval, lookback_minutes)
        if len(ohlc_data) > lookback_minutes + 1:
            ohlc_data = ohlc_data[:lookback_minutes + 1]

        close_prices = [candle['close'] for candle in ohlc_data]
        max_close = max(close_prices[:-1])
        min_close = min(close_prices[:-1])
        newest_close = close_prices[-1]
        newest_candle = ohlc_data[-1]

        # --- Sequence Logic ---
        if sequence is None:
            if newest_close > max_close:
                sequence = "First Break Up Signal"
                historical_events['first_break'] = {'time': newest_candle['time'], 'level': max_close}
                breakout_close = newest_close
                confirmation_candles = []
            elif newest_close < min_close:
                sequence = "First Break Down Signal"
                historical_events['first_break'] = {'time': newest_candle['time'], 'level': min_close}
                breakout_close = newest_close
                confirmation_candles = []

        # --- First Signal Confirmation ---
        if sequence in ["First Break Up Signal", "First Break Down Signal"]:
            confirmation_candles.append(newest_close)
            if len(confirmation_candles) == 3:
                condition_met = (
                    all(c >= breakout_close for c in confirmation_candles) if "Up" in sequence
                    else all(c <= breakout_close for c in confirmation_candles)
                )
                if condition_met:
                    first_signal = "First Bullish Signal" if "Up" in sequence else "First Bearish Signal"
                    reversal_level = min_close if "Up" in sequence else max_close
                    reversal_candle_count = 0
                    historical_events['first_signal'] = {'time': newest_candle['time'], 'level': breakout_close}
                else:
                    sequence = None
                    print("First Signal failed: Next 3 candles invalid.")

        # --- Fast Reversal Break ---
        if first_signal:
            reversal_candle_count += 1
            reversal_condition = (
                (newest_close < reversal_level) if "Bullish" in first_signal
                else (newest_close > reversal_level)
            )
            if reversal_condition:
                sequence = "Fast Reversal Break Down" if "Bullish" in first_signal else "Fast Reversal Break Up"
                historical_events['reversal'] = {'time': newest_candle['time'], 'level': reversal_level}
                confirmation_candles = []
            elif reversal_candle_count >= reversal_candles:
                sequence = None
                print("Reversal failed: Time limit exceeded.")

        # --- Second Break Confirmation ---
        if sequence in ["Fast Reversal Break Down", "Fast Reversal Break Up"]:
            confirmation_candles.append(newest_close)
            if len(confirmation_candles) == 3:
                condition_met = (
                    all(c <= breakout_close for c in confirmation_candles) if "Down" in sequence
                    else all(c >= breakout_close for c in confirmation_candles)
                )
                if condition_met:
                    confirmed_signal = "Confirmed Break Down" if "Down" in sequence else "Confirmed Break Up"
                    confirmed_time = datetime.utcnow()
                    candle_counter = 0

                    # Define zones and TP/SL
                    if confirmed_signal == "Confirmed Break Down":
                        short_zone = {
                            'lower': historical_events['first_break']['level'], 
                            'upper': max([c['high'] for c in ohlc_data[-3:]])
                        }
                        tp_sl_levels = {
                            'SL': short_zone['upper'] * (1 + sl_percent/100),  # Adjustable SL
                            'TP': short_zone['lower'] * (1 - tp_percent/100)   # Adjustable TP
                        }
                    else:
                        long_zone = {
                            'upper': historical_events['first_break']['level'], 
                            'lower': min([c['low'] for c in ohlc_data[-3:]])
                        }
                        tp_sl_levels = {
                            'SL': long_zone['lower'] * (1 - sl_percent/100),  # Adjustable SL
                            'TP': long_zone['upper'] * (1 + tp_percent/100)   # Adjustable TP
                        }

                    # Print Confirmation Details
                    print(f"\n=== {confirmed_signal} CONFIRMED ===")
                    print(f"First Break: {historical_events['first_break']['time']} (Level: {historical_events['first_break']['level']})")
                    print(f"Reversal Break: {historical_events['reversal']['time']} (Level: {historical_events['reversal']['level']})")
                    print(f"Zone: {long_zone if confirmed_signal == 'Confirmed Break Up' else short_zone}")
                    print(f"TP: {tp_sl_levels['TP']:.2f}, SL: {tp_sl_levels['SL']:.2f}")
                    print(f"Time Limit: {time_limit_candles} candles from {confirmed_time.strftime('%Y-%m-%d %H:%M:%S')}\n")

                    # Send Telegram Message
                    message = (
                        f"🚨 *{confirmed_signal} CONFIRMED* 🚨\n"
                        f"First Break: {historical_events['first_break']['time']} (Level: {historical_events['first_break']['level']})\n"
                        f"Reversal Break: {historical_events['reversal']['time']} (Level: {historical_events['reversal']['level']})\n"
                        f"Zone: {long_zone if confirmed_signal == 'Confirmed Break Up' else short_zone}\n"
                        f"TP: {tp_sl_levels['TP']:.2f}, SL: {tp_sl_levels['SL']:.2f}\n"
                        f"Time Limit: {time_limit_candles} candles from {confirmed_time.strftime('%Y-%m-%d %H:%M:%S')}"
                    )
                    send_telegram_message(message)

                else:
                    sequence = None
                    print("Second Break failed: Next 3 candles invalid.")

        # --- TP/SL and Time Limit Check ---
        if confirmed_signal:
            candle_counter += 1

            # Check if price entered the zone
            if not entry_triggered:
                if confirmed_signal == "Confirmed Break Up":
                    entry_triggered = newest_candle['low'] <= long_zone['upper'] and newest_candle['low'] >= long_zone['lower']
                else:
                    entry_triggered = newest_candle['high'] >= short_zone['lower'] and newest_candle['high'] <= short_zone['upper']
                if entry_triggered:
                    print(f"Entry into zone triggered at {newest_candle['time']}")

            # Check TP/SL only if entry triggered
            if entry_triggered:
                if (confirmed_signal == "Confirmed Break Up" and (newest_close >= tp_sl_levels['TP'] or newest_close <= tp_sl_levels['SL'])) or \
                   (confirmed_signal == "Confirmed Break Down" and (newest_close <= tp_sl_levels['TP'] or newest_close >= tp_sl_levels['SL'])):
                    print(f"{'TP' if newest_close >= tp_sl_levels['TP'] else 'SL'} Hit at {newest_candle['time']}!")
                    confirmed_signal = None
                    entry_triggered = False

            # Check time limit
            if candle_counter >= time_limit_candles:
                print(f"Time Limit Reached: {time_limit_candles} candles expired.")
                confirmed_signal = None
                entry_triggered = False

        # --- Print Results ---
        print(f"Time: {newest_candle['time']}")
        print(f"Sequence: {sequence}")
        print(f"Confirmed Signal: {confirmed_signal}")
        print(f"Entry Triggered: {entry_triggered}")
        print(f"Candle Counter: {candle_counter}/{time_limit_candles}")
        print("-" * 40)

# Example Usage
get_minute_info(
    symbol='BTC',
    interval='1m',
    lookback_minutes=100,
    reversal_candles=80,  
    tp_percent=3,         
    sl_percent=3,         
    time_limit_candles=1500
)

/tmp/ipykernel_1871636/2549842297.py:87: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


Waiting 49 seconds until next minute...
Time: 2025-02-28 09:38:00
Sequence: None
Confirmed Signal: None
Entry Triggered: False
Candle Counter: 0/1500
----------------------------------------
Waiting 55 seconds until next minute...
Time: 2025-02-28 09:39:00
Sequence: None
Confirmed Signal: None
Entry Triggered: False
Candle Counter: 0/1500
----------------------------------------
Waiting 55 seconds until next minute...
Time: 2025-02-28 09:40:00
Sequence: None
Confirmed Signal: None
Entry Triggered: False
Candle Counter: 0/1500
----------------------------------------
Waiting 54 seconds until next minute...
Time: 2025-02-28 09:41:00
Sequence: None
Confirmed Signal: None
Entry Triggered: False
Candle Counter: 0/1500
----------------------------------------
Waiting 55 seconds until next minute...
Time: 2025-02-28 09:42:00
Sequence: None
Confirmed Signal: None
Entry Triggered: False
Candle Counter: 0/1500
----------------------------------------
Waiting 55 seconds until next minute...
Time:

KeyboardInterrupt: 